# Market Cap Tracker

Using Crypto compare api

In [ ]:
import requests,json,datetime,time,random, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from urllib.parse import urljoin, urlencode
import csv
import threading


# Path
1) collect volume data

2) process volume data

3) create DF with volume data

4) collect new data

5) process data[i]

6) add new data to previous DF

7) do till 5 sets of data

In [ ]:
def init():
    name = 'VOLUME24HOURTO-%s' % (str(pd.Timestamp.now())[17:19])
    url = 'https://min-api.cryptocompare.com/data/top/volumes?tsym=BTC&limit=1000'
    output = requests.get(url)
    content = json.loads(output.content.decode('utf-8'))
    df = pd.DataFrame(content['Data'])
    df['SYMBOL'] = df['SYMBOL'].str.lower() # added for merging later
    df = df.set_index('SYMBOL')
    df['mean'] = 0
    df[name] = df['VOLUME24HOURTO']
    del df['SUPPLY'],df['VOLUME24HOURTO'],df['ID']
    return df

In [ ]:
def loop(df):
    firstDF = df.copy() #copy previous DF for appending data
    name = 'VOLUME24HOURTO-%s'
    url = 'https://min-api.cryptocompare.com/data/top/volumes?tsym=BTC&limit=1000'
    output = requests.get(url)
    content = json.loads(output.content.decode('utf-8'))
    df = pd.DataFrame(content['Data'])
    #set index to symbol
    df = df.set_index('SYMBOL')
    #change to variable name
    df[name] = df['VOLUME24HOURTO']
    #clean up list before join
    del df['FULLNAME'],df['NAME'],df['ID'],df['VOLUME24HOURTO'],df['SUPPLY']
    firstDF = firstDF.join(df)
    #drop tables when exceeding 27
    if firstDF.shape[1] > 27:
        firstDF = firstDF.drop(firstDF.columns[3],axis=1)
    #calculate mean
    x = firstDF.shape[1]
    meanRow = firstDF.iloc[:,3:x]
    firstDF['mean'] = meanRow.mean(axis=1)
    return firstDF

In [ ]:
def get_all(df):
    # Can be looped hourly, misses 'Mean' column and calculations
    return_df = df.copy()
    all_df = pd.DataFrame()   
    
    for x in range (1,5): # change if you want to get more or less then 1000 results, steps in 250
        # get data
        raw_data = requests.get('https://api.coingecko.com/api/v3/coins/markets?vs_currency=btc&order=market_cap_desc&per_page=250&page={0}&sparkline=false&price_change_percentage=1h%2C24h%2C7d'.format(x))
        json_data = json.loads(raw_data.content)
        df_data = pd.DataFrame(json_data)
        all_df = all_df.append(df_data)

    # copy relevant data into readable df
    columns_list = ['Index', 'Name', 'Rank', 'Market_Cap', 'Price', 'Daily_Perf', 'Volume']
    return_df = pd.DataFrame(columns=columns_list)
    return_df['Index'] = all_df['symbol']
    return_df['Name'] = all_df['name']
    return_df['Rank'] = all_df['market_cap_rank']
    return_df['Market_Cap'] = all_df['market_cap']
    return_df['Price'] = all_df['current_price']
    return_df['Daily_Perf'] = all_df['price_change_24h']
    return_df['Volume'] = all_df['total_volume']
                
    # log volume and price
    vol_col_name = 'log_vol24h_%s' % (str(pd.Timestamp.now())[11:13])
    pri_col_name = 'log_pri24h_%s' % (str(pd.Timestamp.now())[11:13])
    return_df[vol_col_name] = return_df['Volume']
    return_df[pri_col_name] = return_df['Price']
    #return_df = return_df.set_index('Index')

    result = df.merge(return_df,how='right')
    result = result.set_index('Index')
    
    return result    

In [ ]:
def init_df():
    columns_list = ['Index', 'Name', 'Rank', 'Market_Cap', 'Price', 'Daily_Perf', 'Volume']
    return_df = pd.DataFrame(columns=columns_list)
    
    for x in range (1,5): # change if you want to get more or less then 1000 results, steps in 250
        # get data
        raw_data = requests.get('https://api.coingecko.com/api/v3/coins/markets?vs_currency=btc&order=market_cap_desc&per_page=250&page={0}&sparkline=false&price_change_percentage=1h%2C24h%2C7d'.format(x))
        json_data = json.loads(raw_data.content)
        df_data = pd.DataFrame(json_data)
        return_df.loc['Index'] = return_df['Index'].append(df_data['symbol'])
        
    return_df = return_df.set_index('Index')

    return return_df

In [ ]:
def start():
    df = init_df()
    a = get_all(df)

    while True:
        print('Updating DataFrame...')
        a = get_all(a)
        print(a)
        time.sleep(600)
        print('10 min into new loop')
        time.sleep(600)
        print('20 min into new loop')
        time.sleep(600)
        print('30 min into new loop')
        time.sleep(600)
        print('40 min into new loop')
        time.sleep(600)
        print('50 min into new loop')
        time.sleep(600)

In [ ]:
# Run this to print an updated DataFrame every hour 
start()

In [ ]:
b = loop(df)
b